# Stock Market Calendar  

### Past and Future dates when the stock market is open and closed

Produces 3 csv files, namely
1. All dates from 1927-12-30 to 2023-12-31 (all_dates.txt)
2. Dates when stock market is open (all_open_dates.txt)
3. Dates when stock market is closed (all_close_dates.txt)

In [1]:
import datetime
import os

import pandas as pd
import pandas_datareader.data as pdr

In [2]:
start_date = datetime.date(1927 ,12, 30)
end_date   = datetime.date(2023, 12, 31)

### All dates (all_dates.txt)

In [3]:
filename = 'all_dates.txt'
all_dates = []

if not os.path.isfile(filename):
    all_dates = pd.date_range(start_date, end_date)
    all_dates = all_dates.strftime('%Y-%m-%d').to_list()
    with open (filename, 'w') as f:
        for date in all_dates:
            f.write(f'{date}\n')
with open (filename, 'r') as f:
    all_dates = [line.rstrip() for line in f]
all_dates[:10]

['1927-12-30',
 '1927-12-31',
 '1928-01-01',
 '1928-01-02',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-07',
 '1928-01-08']

### Past Open Dates

First we need to get the trading dates from the past.  We'll get the entire timeseries for S&P 500 and assume that its dates are accuracte as to when trading was allowed.

In [4]:
symbol = '^GSPC'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()
timeseries_cache = symbol + '.csv'

if os.path.isfile(timeseries_cache):
    pass
else:
    ts = pdr.DataReader(symbol, 'yahoo', start, end)
    ts.to_csv(timeseries_cache, encoding='utf-8')

ts = pd.read_csv(timeseries_cache, index_col='Date', parse_dates=True)
ts

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,0,17.660000
1928-01-03,17.760000,17.760000,17.760000,17.760000,0,17.760000
1928-01-04,17.719999,17.719999,17.719999,17.719999,0,17.719999
1928-01-05,17.549999,17.549999,17.549999,17.549999,0,17.549999
1928-01-06,17.660000,17.660000,17.660000,17.660000,0,17.660000
...,...,...,...,...,...,...
2021-06-21,4226.240234,4173.399902,4173.399902,4224.790039,3391740000,4224.790039
2021-06-22,4255.839844,4217.270020,4224.609863,4246.439941,3208760000,4246.439941
2021-06-23,4256.600098,4241.430176,4249.270020,4241.839844,3172440000,4241.839844


Get the dates ctock market was open in the past.

In [5]:
# Drop all the columns
ts = ts.drop(ts.columns, axis=1)
past_open_dates = ts.index.strftime('%Y-%m-%d').to_list()
past_open_dates[:10]

['1927-12-30',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-09',
 '1928-01-10',
 '1928-01-11',
 '1928-01-12',
 '1928-01-13']

### Future Open Dates

We will use a list of the US holiday schedule for 2021-2023.  We take those days away from complete business calander (Mon through Fri) for 2021-2023.

List of dates market is closed in 2021-2023 for US holidays:¶

https://www.nyse.com/markets/hours-calendars


In [6]:
future_holidays = [
    # 2021
    '2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31',
    '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24',
    # 2022
    '2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30', '2022-07-04',
    '2022-09-05', '2022-11-24', '2022-12-26',
    # 2023       
    '2023-01-02', '2023-01-16', '2023-01-20', '2023-04-07', '2023-05-29',
    '2023-07-04', '2023-09-04', '2023-11-23', '2023-12-05'
]

Get list of future business dates.

In [7]:
start = datetime.date(2021 ,1, 1)
end   = datetime.date(2023, 12, 31)

future_business_dates = pd.date_range(start, end, freq='B')
future_business_dates = future_business_dates.strftime('%Y-%m-%d').to_list()
future_business_dates[:10]

['2021-01-01',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14']

Remove the holidays from the business dates and that will be the future market open dates.

In [8]:
future_open_dates = list(set(future_business_dates) - set(future_holidays))
future_open_dates = sorted(future_open_dates)
future_open_dates[:10]

['2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15']

### All Open Dates

Finally, join the past and future market open lists.

In [9]:
all_open_dates = set(past_open_dates).union(set(future_open_dates))
all_open_dates = sorted(all_open_dates)
all_open_dates[:10]

['1927-12-30',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-09',
 '1928-01-10',
 '1928-01-11',
 '1928-01-12',
 '1928-01-13']

In [10]:
filename = 'all_open_dates.txt'
with open (filename, 'w') as f:
    for date in all_open_dates:
        f.write(f'{date}\n')

### All Close Dates

In [11]:
all_close_dates = list(set(all_dates) - set(all_open_dates))
all_close_dates = sorted(all_close_dates)
all_close_dates[:10]

['1927-12-31',
 '1928-01-01',
 '1928-01-02',
 '1928-01-07',
 '1928-01-08',
 '1928-01-14',
 '1928-01-15',
 '1928-01-21',
 '1928-01-22',
 '1928-01-28']

In [12]:
filename = 'all_close_dates.txt'
with open (filename, 'w') as f:
    for date in all_close_dates:
        f.write(f'{date}\n')